# Clean zip code data by convertin shapefiles

In [9]:
import requests
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import seaborn as sns

font = {'family' : 'Arial',
        'weight' : 'bold',
        'size'   : 25}
matplotlib.rc('font', **font)

# Summary
1. Get the shapefiles: easy
2. Select cities (50 largest cities): normal
3. Get the business data: easy
4. Select columns: difficult
5. Get the census data (2010, 2011, 2012, 2013): easy

# 1. Get the shapefiles by zip code

[US census](https://www.census.gov/geo/maps-data/data/cbf/cbf_zcta.html) provides shapefiles for every zip code in the nation. Typically, many people use [this website](http://ogre.adc4gis.com/). However, US census file is very big, and the website cannot handle it. [This page](http://ben.balter.com/2013/06/26/how-to-convert-shapefiles-to-geojson-for-use-on-github/) provides solutions:
1. Install `gdal`
2. Run `$ ogr2ogr -f GeoJSON -t_srs crs:84 [name].geojson [name].shp` where `[name]` is the name of the file. Make sure that you have all the unzipped shapefiles with `[name].shp` in the same folder.

In [10]:
# Check if this can be rendered
import folium
m = folium.Map(location=[37.769959, -122.448679], zoom_start=9)
geo_path = 'zip.geojson'
m.geo_json(geo_path= geo_path)
m.create_map('zip.html')

# 2. Get the city zip codes
1. Get the list of major cities.
2. Get the name

In [11]:
# list of major 50 cities
df_ct = pd.read_csv('citi_list.csv')
df_ct.head()

,city,state,zipcodes
0,Baltimore,MD,"21201, 21202, 21205, 21206, 21207, 21208, 2120..."
1,Boston,MA,"02108, 02109, 02110, 02111, 02113, 02114, 0211..."
2,Chicago,IL,"60018, 60068, 60176, 60601, 60602, 60603, 6060..."
3,Detroit,MI,"48201, 48202, 48203, 48204, 48205, 48206, 4820..."
4,Los Angeles,CA,"90003, 90004, 90005, 90006, 90008, 90007, 9001..."


In [12]:
# There are many zip codes in a city, so we split the zipcodes and replicate the rows
df_city = pd.concat([pd.DataFrame([df_ct['city'].values[i]]*len(df_ct['zipcodes'][i].split(', '))) for i in range(len(df_ct))],ignore_index=True)
df_state = pd.concat([pd.DataFrame([df_ct['state'].values[i]]*len(df_ct['zipcodes'][i].split(', '))) for i in range(len(df_ct))],ignore_index=True)
df_zipcodes = pd.concat([pd.DataFrame(df_ct['zipcodes'][i].split(', ')) for i in range(len(df_ct))],ignore_index=True)

In [13]:
# put them together and name the columns
df_city_zip = pd.concat([df_city,df_state,df_zipcodes],axis=1)
df_city_zip.columns = df_ct.columns

In [14]:
df_city_zip.head()

,city,state,zipcodes
0,Baltimore,MD,21201
1,Baltimore,MD,21202
2,Baltimore,MD,21205
3,Baltimore,MD,21206
4,Baltimore,MD,21207


In [17]:
df_city_zip.to_csv('city_zip_df.csv',encoding='utf-8',index=False)

# 3. Sanity check: compare the zip code list w/ ZCTA zipcodes

In [18]:
# open zip code file
import json
with open('zip.geojson') as f:
    data = json.load(f)

In [19]:
# Get shapfile zipcode list
zcta_zipcodes = []
for i in range(len(data['features'])):
    zcta_zipcodes.append(str(data['features'][i]['properties']['ZCTA5CE10']))

In [20]:
# Get the zipcode, which only exists in ZCTA
zipcodes_filtered = []
for zipcode in df_city_zip.zipcodes:
    if zipcode in zcta_zipcodes:
        zipcodes_filtered.append(zipcode)

# 4. Trim the geojson file to get the selected cities

In [21]:
""" 
    'data' has a key 'feature' which has n rows, 
    each of which has 'properties' key which has 'ZCTA5CE10'
"""
df_feat = pd.DataFrame(data['features'])

In [22]:
df_feat.head()

,geometry,properties,type
0,"{u'type': u'Polygon', u'coordinates': [[[-87.3...","{u'AWATER10': 12694318, u'AFFGEOID10': u'86000...",Feature
1,"{u'type': u'Polygon', u'coordinates': [[[-110....","{u'AWATER10': 488785, u'AFFGEOID10': u'8600000...",Feature
2,"{u'type': u'Polygon', u'coordinates': [[[-109....","{u'AWATER10': 3082, u'AFFGEOID10': u'8600000US...",Feature
3,"{u'type': u'Polygon', u'coordinates': [[[-90.8...","{u'AWATER10': 10549055, u'AFFGEOID10': u'86000...",Feature
4,"{u'type': u'Polygon', u'coordinates': [[[-118....","{u'AWATER10': 1540929, u'AFFGEOID10': u'860000...",Feature


In [23]:
# create a new df (temp) for selected cities
temp = pd.DataFrame()
for i in range(len(data['features'])):
    if str(data['features'][i]['properties']['ZCTA5CE10']) in zipcodes_filtered:
        temp = pd.concat([temp,df_feat.iloc[[i]]],ignore_index=True)
len(temp)

586

In [24]:
temp.head()

,geometry,properties,type
0,"{u'type': u'Polygon', u'coordinates': [[[-122....","{u'AWATER10': 1299131, u'AFFGEOID10': u'860000...",Feature
1,"{u'type': u'Polygon', u'coordinates': [[[-77.0...","{u'AWATER10': 0, u'AFFGEOID10': u'8600000US205...",Feature
2,"{u'type': u'Polygon', u'coordinates': [[[-73.9...","{u'AWATER10': 0, u'AFFGEOID10': u'8600000US112...",Feature
3,"{u'type': u'Polygon', u'coordinates': [[[-74.0...","{u'AWATER10': 0, u'AFFGEOID10': u'8600000US102...",Feature
4,"{u'type': u'Polygon', u'coordinates': [[[-73.9...","{u'AWATER10': 0, u'AFFGEOID10': u'8600000US101...",Feature


In [25]:
# compare the structure of the original data and the df so that we don't corrupt geojson structure
"""
    Original
"""
from pprint import pprint
pprint(data['features'][0])

{u'geometry': {u'coordinates': [[[-87.390128, 31.26047, 0.0],
                                 [-87.365618, 31.260576, 0.0],
                                 [-87.36528, 31.260364, 0.0],
                                 [-87.339969, 31.260608, 0.0],
                                 [-87.316832, 31.260845, 0.0],
                                 [-87.314535, 31.260712, 0.0],
                                 [-87.22548, 31.260639, 0.0],
                                 [-87.205088, 31.276148, 0.0],
                                 [-87.207538, 31.279942, 0.0],
                                 [-87.209069, 31.281372, 0.0],
                                 [-87.213412, 31.28365, 0.0],
                                 [-87.215381, 31.285203, 0.0],
                                 [-87.216622, 31.288821, 0.0],
                                 [-87.217267, 31.294189, 0.0],
                                 [-87.218934, 31.29895, 0.0],
                                 [-87.220058, 31.305251, 0.0

In [26]:
"""
    Trimmed: 
"""
# same as the original
dict_temp = temp.to_dict(orient='records')
pprint(dict_temp[0])

{u'geometry': {u'coordinates': [[[-122.50808771796498,
                                  37.73519779053799,
                                  0.0],
                                 [-122.502558, 37.735557, 0.0],
                                 [-122.496449, 37.733877, 0.0],
                                 [-122.493784, 37.733989, 0.0],
                                 [-122.491294, 37.734096, 0.0],
                                 [-122.49146, 37.737283, 0.0],
                                 [-122.490364, 37.73793, 0.0],
                                 [-122.488518, 37.737002, 0.0],
                                 [-122.485767, 37.736757, 0.0],
                                 [-122.483971, 37.737387, 0.0],
                                 [-122.481727, 37.737172, 0.0],
                                 [-122.47539, 37.73745, 0.0],
                                 [-122.475221, 37.734869, 0.0],
                                 [-122.471569, 37.734707, 0.0],
                        

In [27]:
# replace the original
data['features'] = dict_temp

In [28]:
# save as a new file
import json
with open('zip_cities.json', 'w') as f:
    json.dump(data, f)

In [29]:
# test using folium
m = folium.Map(location=[37.769959, -122.448679], zoom_start=9)
geo_path = 'zip_cities.json'
m.geo_json(geo_path= geo_path)
m.create_map('zip_cities.html')

In [31]:
# save the zip code file
import pickle
with open('zipcode_final.txt', 'wb') as f:
    pickle.dump(zipcodes_filtered, f)

In [ ]:
# # load the file 
# with open('zipcode_final.txt', 'rb') as f:
#     zip_list = pickle.load(f)    